In [16]:
#Importing Required libraries
import pandas as pd
import numpy as np
import math
import sys
import findspark
findspark.init()

In [30]:
from pyspark import StorageLevel
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, lit, col, explode, initcap, regexp_replace, split, concat, substring, to_date, when,count
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DoubleType
from pyspark.sql.dataframe import DataFrame

In [31]:
spark = SparkSession \
 .builder \
 .appName("PlaysoreAppsPySpark") \
 .getOrCreate()

In [32]:
# Reading the raw dataset from Cloud Storage bucketin to the python dataframe psa
psapps_df = spark.read.csv("gs://ca675-assignment2-bucket/Raw_Data/Google-Playstore.csv",header='true',escape='"',inferSchema='true')
# Checking the first 10 rows of the dataframe
psapps_df.show(10)

+--------------------+--------------------+----------------+------+------------+--------+----------------+----------------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+------------+------------+--------------+--------------------+------------+----------------+--------------+-------------------+
|            App Name|              App Id|        Category|Rating|Rating Count|Installs|Minimum Installs|Maximum Installs|Free|Price|Currency|Size|Minimum Android|        Developer Id|   Developer Website|     Developer Email|    Released|Last Updated|Content Rating|      Privacy Policy|Ad Supported|In App Purchases|Editors Choice|       Scraped Time|
+--------------------+--------------------+----------------+------+------------+--------+----------------+----------------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+------------+------------+--------------+--------------------+-

In [33]:
#Checking the structure of the numeric feature
psapps_df.describe()


DataFrame[summary: string, App Name: string, App Id: string, Category: string, Rating: string, Rating Count: string, Installs: string, Minimum Installs: string, Maximum Installs: string, Price: string, Currency: string, Size: string, Minimum Android: string, Developer Id: string, Developer Website: string, Developer Email: string, Released: string, Last Updated: string, Content Rating: string, Privacy Policy: string, Scraped Time: string]

In [34]:
#Checking the total count null values in eah feature of the dataframe psapps_df
psapps_null_df=psapps_df.select([count(when(col(c).isNull(),c)) for c in psapps_df.columns])
psapps_null_df.show()

+-----------------------------------------------------+-------------------------------------------------+-----------------------------------------------------+-------------------------------------------------+-------------------------------------------------------------+-----------------------------------------------------+---------------------------------------------------------------------+---------------------------------------------------------------------+---------------------------------------------+-----------------------------------------------+-----------------------------------------------------+---------------------------------------------+-------------------------------------------------------------------+-------------------------------------------------------------+-----------------------------------------------------------------------+-------------------------------------------------------------------+-----------------------------------------------------+-----------------

In [35]:
#Cleaning-Step1
#Removing the unwanted features abd storing the resultant in new dataframe 'psapp
psapps_selected_df=psapps_df.drop("App Id","Minimum Installs","Currency","Minimum Android","Developer Id","Developer Website","Last Updated","Privacy Policy","Editors Choice","Scraped Time")
# Checking top 10 values after removal of unwanted features
psapps_selected_df.head(10)


[Row(App Name='Gakondo', Category='Adventure', Rating=0.0, Rating Count=0, Installs='10+', Maximum Installs=15, Free=True, Price=0.0, Size='10M', Developer Email='jean21101999@gmail.com', Released='Feb 26, 2020', Content Rating='Everyone', Ad Supported=False, In App Purchases=False),
 Row(App Name='Ampere Battery Info', Category='Tools', Rating=4.4, Rating Count=64, Installs='5,000+', Maximum Installs=7662, Free=True, Price=0.0, Size='2.9M', Developer Email='webserveis@gmail.com', Released='May 21, 2020', Content Rating='Everyone', Ad Supported=True, In App Purchases=False),
 Row(App Name='Vibook', Category='Productivity', Rating=0.0, Rating Count=0, Installs='50+', Maximum Installs=58, Free=True, Price=0.0, Size='3.7M', Developer Email='vnacrewit@gmail.com', Released='Aug 9, 2019', Content Rating='Everyone', Ad Supported=False, In App Purchases=False),
 Row(App Name='Smart City Trichy Public Service Vehicles 17UCS548', Category='Communication', Rating=5.0, Rating Count=5, Installs='10

In [36]:
#Cleaning-Step2
# Replacing the space in column names with '_' and storing the new fieldnames in 
df = [col(column).alias(column.replace(' ', '_')) for column in psapps_selected_df.columns]
psapps_selected_df.select(*df).show()
df1=psapps_selected_df.select(*df)


+----------------------------------+-----------------+------+------------+--------+----------------+----+-----+----+--------------------+------------+--------------+------------+----------------+
|                          App_Name|         Category|Rating|Rating_Count|Installs|Maximum_Installs|Free|Price|Size|     Developer_Email|    Released|Content_Rating|Ad_Supported|In_App_Purchases|
+----------------------------------+-----------------+------+------------+--------+----------------+----+-----+----+--------------------+------------+--------------+------------+----------------+
|                           Gakondo|        Adventure|   0.0|           0|     10+|              15|true|  0.0| 10M|jean21101999@gmai...|Feb 26, 2020|      Everyone|       false|           false|
|               Ampere Battery Info|            Tools|   4.4|          64|  5,000+|            7662|true|  0.0|2.9M|webserveis@gmail.com|May 21, 2020|      Everyone|        true|           false|
|                   

In [37]:
# Checking data structure of the dataframe df1
df1.printSchema()


root
 |-- App_Name: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Rating_Count: integer (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Maximum_Installs: long (nullable = true)
 |-- Free: boolean (nullable = true)
 |-- Price: double (nullable = true)
 |-- Size: string (nullable = true)
 |-- Developer_Email: string (nullable = true)
 |-- Released: string (nullable = true)
 |-- Content_Rating: string (nullable = true)
 |-- Ad_Supported: boolean (nullable = true)
 |-- In_App_Purchases: boolean (nullable = true)



In [38]:
#Cleaning-Step3
# Changing the datatype of Rating column to Double, Installs to Integer, Price to
df1 = df1 \
 .withColumn("Rating", col("Rating").cast(DoubleType())) \
 .withColumn("Rating_Count", col("Rating_Count").cast(IntegerType())) \
 .withColumn("Installs", regexp_replace(col("Installs"), "[^0-9]", "")) \
 .withColumn("Installs", col("Installs").cast(IntegerType())) \
 .withColumn("Price", regexp_replace(col("Price"), "[$]", "")) \
 .withColumn("Price", col("Price").cast(DoubleType())) \
 .withColumn("Released", to_date('Released', 'MMM d, yyyy'))


In [39]:
#Checking data structure of the dataframe df1
df1.printSchema()

root
 |-- App_Name: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Rating_Count: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Maximum_Installs: long (nullable = true)
 |-- Free: boolean (nullable = true)
 |-- Price: double (nullable = true)
 |-- Size: string (nullable = true)
 |-- Developer_Email: string (nullable = true)
 |-- Released: date (nullable = true)
 |-- Content_Rating: string (nullable = true)
 |-- Ad_Supported: boolean (nullable = true)
 |-- In_App_Purchases: boolean (nullable = true)



In [40]:
#Cleaning-Step4
# Dropping rows having null values in Release column and storing the result in dataframe df2
df2=df1.dropna(subset=["Released"])
df2.show(10)


+--------------------+----------------+------+------------+--------+----------------+----+-----+----+--------------------+----------+--------------+------------+----------------+
|            App_Name|        Category|Rating|Rating_Count|Installs|Maximum_Installs|Free|Price|Size|     Developer_Email|  Released|Content_Rating|Ad_Supported|In_App_Purchases|
+--------------------+----------------+------+------------+--------+----------------+----+-----+----+--------------------+----------+--------------+------------+----------------+
|             Gakondo|       Adventure|   0.0|           0|      10|              15|true|  0.0| 10M|jean21101999@gmai...|2020-02-26|      Everyone|       false|           false|
| Ampere Battery Info|           Tools|   4.4|          64|    5000|            7662|true|  0.0|2.9M|webserveis@gmail.com|2020-05-21|      Everyone|        true|           false|
|              Vibook|    Productivity|   0.0|           0|      50|              58|true|  0.0|3.7M| vna

In [41]:
#Checking the total count null values in eah feature of the dataframe df2
df2_null1 = df2.select([count(when(col(c).isNull(),c)) for c in df2.columns])
df2_null1.show()


+-----------------------------------------------------+-----------------------------------------------------+-------------------------------------------------+-------------------------------------------------------------+-----------------------------------------------------+---------------------------------------------------------------------+---------------------------------------------+-----------------------------------------------+---------------------------------------------+-------------------------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------------------+-------------------------------------------------------------+---------------------------------------------------------------------+
|count(CASE WHEN (App_Name IS NULL) THEN App_Name END)|count(CASE WHEN (Category IS NULL) THEN Category END)|count(CASE WHEN (Rating IS NULL) THEN Rating END)|count(CASE WHEN (Rating_Count IS NU

In [42]:
#Cleaning-Step5
# Dropping rows having null values in Installs column and storing the result in d
df3=df2.na.drop(subset=["Installs"])
df3.show(10)

+--------------------+----------------+------+------------+--------+----------------+----+-----+----+--------------------+----------+--------------+------------+----------------+
|            App_Name|        Category|Rating|Rating_Count|Installs|Maximum_Installs|Free|Price|Size|     Developer_Email|  Released|Content_Rating|Ad_Supported|In_App_Purchases|
+--------------------+----------------+------+------------+--------+----------------+----+-----+----+--------------------+----------+--------------+------------+----------------+
|             Gakondo|       Adventure|   0.0|           0|      10|              15|true|  0.0| 10M|jean21101999@gmai...|2020-02-26|      Everyone|       false|           false|
| Ampere Battery Info|           Tools|   4.4|          64|    5000|            7662|true|  0.0|2.9M|webserveis@gmail.com|2020-05-21|      Everyone|        true|           false|
|              Vibook|    Productivity|   0.0|           0|      50|              58|true|  0.0|3.7M| vna

In [ ]:
#Checking the total count null values in eah feature of the dataframe df3
df3_null1 = df3.select([count(when(col(c).isNull(),c)) for c in df3.columns])
df3_null1.show()


In [52]:
#Cleaning-Step6
# Modifying the values of Size Column in dataframe df3
df3 = df3 \
 .withColumn("Size", regexp_replace(col("Size"), 'k', 'e+3')) \
 .withColumn("Size", regexp_replace(col("Size"), 'M', 'e+6')) \
 .withColumn("Size", regexp_replace(col("Size"), 'G', 'e+9')) \
 .withColumn("Size", regexp_replace(col("Size"), "Varies with Device", 'nan')) \
 .withColumn("Size", regexp_replace(col("Size"), "[+]", "")) \
 .withColumn("Size", regexp_replace(col("Size"), "[^0-9]", "")) \
 .withColumn("Size", col("Size").cast(IntegerType()))


In [53]:
#Checking the total count nan values in Size feature of the dataframe df3
df3.select([count(when(isnan('Size'),True))]).show()


+------------------------------------------+
|count(CASE WHEN isnan(Size) THEN true END)|
+------------------------------------------+
|                                         0|
+------------------------------------------+



In [54]:
# Checking the data structure of dataframe df3
df3.printSchema()


root
 |-- App_Name: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Rating_Count: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Maximum_Installs: long (nullable = true)
 |-- Free: boolean (nullable = true)
 |-- Price: double (nullable = true)
 |-- Size: integer (nullable = true)
 |-- Developer_Email: string (nullable = true)
 |-- Released: date (nullable = true)
 |-- Content_Rating: string (nullable = true)
 |-- Ad_Supported: boolean (nullable = true)
 |-- In_App_Purchases: boolean (nullable = true)



In [55]:
#Cleaning-Step7
# Dropping rows having null values in columns Rating and Rating_Count
df4=df3.na.drop(subset=["Rating", "Rating_Count"])
df4.show(10)

+--------------------+----------------+------+------------+--------+----------------+----+-----+----+--------------------+----------+--------------+------------+----------------+
|            App_Name|        Category|Rating|Rating_Count|Installs|Maximum_Installs|Free|Price|Size|     Developer_Email|  Released|Content_Rating|Ad_Supported|In_App_Purchases|
+--------------------+----------------+------+------------+--------+----------------+----+-----+----+--------------------+----------+--------------+------------+----------------+
|             Gakondo|       Adventure|   0.0|           0|      10|              15|true|  0.0| 106|jean21101999@gmai...|2020-02-26|      Everyone|       false|           false|
| Ampere Battery Info|           Tools|   4.4|          64|    5000|            7662|true|  0.0| 296|webserveis@gmail.com|2020-05-21|      Everyone|        true|           false|
|              Vibook|    Productivity|   0.0|           0|      50|              58|true|  0.0| 376| vna

In [56]:
#Checking the total count null values in eah feature of the dataframe df4
df4_null1 = df4.select([count(when(col(c).isNull(),c)) for c in df4.columns])
df4_null1.show()


+-----------------------------------------------------+-----------------------------------------------------+-------------------------------------------------+-------------------------------------------------------------+-----------------------------------------------------+---------------------------------------------------------------------+---------------------------------------------+-----------------------------------------------+---------------------------------------------+-------------------------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------------------+-------------------------------------------------------------+---------------------------------------------------------------------+
|count(CASE WHEN (App_Name IS NULL) THEN App_Name END)|count(CASE WHEN (Category IS NULL) THEN Category END)|count(CASE WHEN (Rating IS NULL) THEN Rating END)|count(CASE WHEN (Rating_Count IS NU

In [57]:
#Cleaning-Step8
# Dropping rows having null values in column Size
df5=df4.na.drop(subset=["Size"])
df5.show(10)


+--------------------+----------------+------+------------+--------+----------------+----+-----+----+--------------------+----------+--------------+------------+----------------+
|            App_Name|        Category|Rating|Rating_Count|Installs|Maximum_Installs|Free|Price|Size|     Developer_Email|  Released|Content_Rating|Ad_Supported|In_App_Purchases|
+--------------------+----------------+------+------------+--------+----------------+----+-----+----+--------------------+----------+--------------+------------+----------------+
|             Gakondo|       Adventure|   0.0|           0|      10|              15|true|  0.0| 106|jean21101999@gmai...|2020-02-26|      Everyone|       false|           false|
| Ampere Battery Info|           Tools|   4.4|          64|    5000|            7662|true|  0.0| 296|webserveis@gmail.com|2020-05-21|      Everyone|        true|           false|
|              Vibook|    Productivity|   0.0|           0|      50|              58|true|  0.0| 376| vna

In [58]:
#Checking the total count null values in eah feature of the dataframe df5
df5_null1 = df5.select([count(when(col(c).isNull(),c)) for c in df5.columns])
df5_null1.show()


+-----------------------------------------------------+-----------------------------------------------------+-------------------------------------------------+-------------------------------------------------------------+-----------------------------------------------------+---------------------------------------------------------------------+---------------------------------------------+-----------------------------------------------+---------------------------------------------+-------------------------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------------------+-------------------------------------------------------------+---------------------------------------------------------------------+
|count(CASE WHEN (App_Name IS NULL) THEN App_Name END)|count(CASE WHEN (Category IS NULL) THEN Category END)|count(CASE WHEN (Rating IS NULL) THEN Rating END)|count(CASE WHEN (Rating_Count IS NU

In [59]:
df5.count()

2190445